In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from inference_toolbox.parameter import Parameter
from inference_toolbox.model import Model
from inference_toolbox.likelihood import Likelihood
from inference_toolbox.sampler import Sampler
from inference_toolbox.visualiser import Visualiser
from inference_toolbox.domain import Domain

from data_processing.get_data import get_data
import os

current_directory = os.getcwd()
if current_directory != '/project/':
    os.chdir('/project/')

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [2]:
data_type = 'dummy'
data_params = {
    'data_type': 'dummy',
    'data_path': 'data',
    'sigma': 1,
    'model_select': 'log_gpm_alt_norm',
    'noise_dist': 'gaussian',
    'model': {
        'model_params':{
            'H': 10
        },
        'inference_params':{
            'I_y': 0.1,
            'I_z': 0.1,
            'Q': 3e13,
        },
    },
    'domain': {
        'domain_select': 'cone_from_source_z_limited', 
        'resolution': 20,
        'domain_params':{
            'r': 100,
            'theta': np.pi/8,
            'source': [0,0,10]}
    },
    'output_header': 'Concentration'
}

data_type = 'gridded'
data_params = {
    'data_type': 'gridded',
    'output_header': 'Concentration',
    'log':True,
    'grid_size': [200,200,50],
    'target': False,
    'data_path':'data'
}

data = get_data(data_type, data_params)

In [3]:
# training_data, testing_data = train_test_split(averaged_df, test_size=0.2)
training_data, testing_data = train_test_split(data, test_size=0.2)

In [4]:
# Initialize parameter series
params = pd.Series({},dtype='float64')

# Parameter Assignment
I_y = Parameter(name = 'I_y', prior_select="gamma")
I_y.add_prior_param("mu",5.2640)
I_y.add_prior_param("sigma",5.3316)
params['I_y'] = I_y

I_z = Parameter(name = 'I_z', prior_select="gamma")
I_z.add_prior_param("mu",1.7125)
I_z.add_prior_param("sigma",2.6842)
params['I_z'] = I_z

Q = Parameter(name = 'Q', prior_select="gamma")
Q.add_prior_param("mu",5.0986e16)
Q.add_prior_param("sigma",6.5244e17)
params['Q'] = Q

sigma = Parameter(name = 'sigma', prior_select="gamma")
sigma.add_prior_param("mu",1.2003)
sigma.add_prior_param("sigma", 1.8308)
params['sigma'] = sigma

# Model Assignment
model = Model('log_gpm_alt_norm')
model.add_model_param("H",10)

# Likelihood function assigmnent
likelihood = Likelihood("gaussian")
# likelihood.add_likelihood_param("sigma",1)
# likelihood.add_likelihood_param("lambda_1",1)
# likelihood.add_likelihood_param("lambda_2",0.05)

num_samples = 10000

data_path = 'results/gridded_drone_data_2/inference/general_instances'

# Initialize the sampler
sampler = Sampler(params, 
                  model, 
                  likelihood, 
                  training_data, 
                  testing_data,
                  num_samples, 
                  show_sample_info = True, 
                  n_chains=3, 
                  thinning_rate=1,
                  data_path = data_path)

# Sample the parameters
params_samples, chain_samples, fields = sampler.sample_all()

/project/inference_toolbox/sampler.py:59: UserWarning: There are not enough devices to run parallel chains: expected 3 but got 1. Chains will be drawn sequentially. If you are running MCMC in CPU, consider using `numpyro.set_host_device_count(3)` at the beginning of your program. You can double-check how many devices are available in your system using `jax.local_device_count()`.
  self.mcmc_obj = numpyro.infer.MCMC(kernel, num_warmup=self.n_warmup, num_samples=self.n_samples, num_chains=self.n_chains, thinning=self.thinning_rate)
sample: 100%|██████████| 12500/12500 [00:01<00:00, 6463.17it/s, 3 steps of size 2.26e-01. acc. prob=0.93] 


In [5]:
# visualiser = Visualiser(testing_data, 
#                         params_samples, 
#                         model, 
#                         sampler.hyperparams, 
#                         chain_samples=chain_samples,
#                         fields = fields, 
#                         previous_instance = sampler.instance, 
#                         data_path = data_path, 
#                         actual_values=[
#                             data_params['model']['inference_params']['I_y'], 
#                             data_params['model']['inference_params']['I_z'],
#                             data_params['model']['inference_params']['Q'],
#                             data_params['sigma']])

visualiser = Visualiser(testing_data, 
                        sampler,
                        model, 
                        previous_instance = sampler.instance, 
                        data_path = data_path)

visualiser.get_summary()
visualiser.get_traceplot()
visualiser.get_autocorrelations()
print('Log Likelihood: ', visualiser.log_likelihood)
print('AIC: ', visualiser.AIC)
print('RMSE: ', visualiser.RMSE)

domain = Domain('cone_from_source_z_limited', resolution=80)
domain.add_domain_param('r', 100)
domain.add_domain_param('theta', np.pi/8)
domain.add_domain_param('source', [0,0,10])

visualiser.visualise_results(domain = domain, name = 'small_scale_3D_plots', title='Log Concentration of Droplets', log_results=False)
visualiser.animate(name = 'small_scale_3D_plots')

Creating instance
Log Likelihood:  -14.270524
AIC:  22.27052402496338
RMSE:  0.41333362
